## Selenium Extraction 

### Local Imports

In [2]:
import os
import re
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains

# Text matching
from difflib import SequenceMatcher

#### Parameters

In [3]:
URL = "https://www.chip.gov.co/schip_rt/index.jsf"
FILE_PATH = "C:/Users/fdmol/Desktop/upwork_projects/DIVIPOLA_Municipios.xlsx"
TIMEOUT = 10
OUTPUT_PATH = "C:/Users/fdmol/Desktop/upwork_projects/outputs"

### Functions and setup

In [4]:
def read_from_xls(FILE_PATH):
    """
    Reads provided xls file and returns a dataframe
    """
    THRESHOLD = 4
    INVALID_ROWS = 2
    df = pd.read_excel(FILE_PATH)

    # Drop rows with empty values
    df = df.dropna(thresh=THRESHOLD).reset_index(drop=True)
    # Ignore First two row
    df = df.iloc[INVALID_ROWS:].reset_index(drop=True).copy()

    # Rename columns
    df.columns = [
        "departamento_codigo",
        "departamento_nombre",
        "municipio_codigo",
        "municipio_nombre",
        "tipo",
        "latitud",
        "longitud",
    ]

    return df

In [5]:
def go_to_consultas(driver):
    """
    Goes to consulta ciudadanda page
    """
    consulta_xpath = '//*[@id="j_idt17:j_idt19:j_idt32"]/a/span'
    informe_xpath = '//*[@id="j_idt17:j_idt19:j_idt32:InformacionEnviada:out"]'
    driver.get(URL)
    print(f"Called {URL}")
    WebDriverWait(driver, TIMEOUT).until(
        EC.presence_of_element_located((By.XPATH, consulta_xpath))
    )
    try:
        driver.find_element(by=By.XPATH, value=consulta_xpath).click()
    except Exception as e:
        print(e)

        driver.execute_script(
            "arguments[0].click();", driver.find_element(By.XPATH, consulta_xpath)
        )
    time.sleep(2)
    WebDriverWait(driver, TIMEOUT).until(
        EC.presence_of_element_located((By.XPATH, informe_xpath))
    )
    try:
        driver.find_element(by=By.XPATH, value=informe_xpath).click()

    except Exception as e:
        print(e)
        driver.execute_script(
            "arguments[0].click();", driver.find_element(By.XPATH, informe_xpath)
        )
    print(f"Entered consulta ciudadana page")

In [6]:
def get_list_of_options(driver):
    """
    Gets possible options for the dropdown menu
    """
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    clean_names = []
    divs = soup.find(
        "div",
        {
            "id": "frm1:SelBoxEntidadCiudadano_div",
            "class": "ui-widget ui-widget-content ui-corner-all",
        },
    )

    sub_divs = divs.find_all("div")
    if sub_divs:
        for sub_div in sub_divs:
            raw_text = sub_div.text
            # Keep words only, not numbers
            clean_text = re.sub(r"\d+\W+", "", raw_text)
            # Remove numbers
            clean_names.append(clean_text)

    return clean_names

In [7]:
def get_best_match(list_names, name_municipio):
    """
    Gets best match for a given name_municipio in a list of names
    """

    def similar(a, b):
        return SequenceMatcher(None, a, b).ratio()

    similarity_ratios = []

    # REMOVE ACCENTS
    mapping_dict = {
        "Á": "A",
        "É": "E",
        "Í": "I",
        "Ó": "O",
        "Ú": "U",
    }

    for key, value in mapping_dict.items():
        name_municipio = name_municipio.replace(key, value)

    list_names = [str(name).upper() for name in list_names]
    for name in list_names:
        similarity_ratios.append(similar(name_municipio.upper(), name.upper()))

    max_ratio = max(similarity_ratios)
    idx_max_ratio = similarity_ratios.index(max_ratio)
    print(idx_max_ratio)
    print(
        f"Max Ratio: {max_ratio} for {name_municipio} and {list_names[idx_max_ratio]}"
    )

    return idx_max_ratio

In [8]:
def get_desired_category_xpath(driver):
    """Returns the xpath of the desired category in the dropdown menu"""

    desired_category_xpath = '//*[@id="frm1:SelBoxCategoria"]/option[num]'
    soup = BeautifulSoup(driver.page_source, "html.parser")
    option_tags = soup.find_all("option")
    counter = 0
    success = False
    for option_tag in option_tags:
        text = option_tag.text
        if "FUT_INGRESO" in text:
            counter += 1
            success = True
            break

        counter += 1

    if success:
        desired_category_xpath = desired_category_xpath.replace("num", str(counter))
    else:
        desired_category_xpath = None

    return desired_category_xpath

In [9]:
def select_entidad_and_categoria(municipio, driver):
    """
    Selects entity and category from the dropdowns
    """
    entidad_xpath = '//*[@id="frm1:SelBoxEntidadCiudadano_input"]'
    hidden_table_xpath = '//*[@id="frm1:SelBoxEntidadCiudadano_div"]'
    desired_option = '//*[@id="frm1:SelBoxEntidadCiudadano_div"]/div/div[num]'

    WebDriverWait(driver, TIMEOUT).until(
        EC.element_to_be_clickable((By.XPATH, entidad_xpath))
    )
    entidad = driver.find_element(by=By.XPATH, value=entidad_xpath)

    mapping_dict = {
        "Á": "A",
        "É": "E",
        "Í": "I",
        "Ó": "O",
        "Ú": "U",
    }

    for key, value in mapping_dict.items():
        municipio = municipio.replace(key, value)

    entidad.send_keys(municipio)
    time.sleep(1.25)

    options = get_list_of_options(driver)
    correct_index = get_best_match(options, municipio)

    desired_option = desired_option.replace("num", str(correct_index))
    print(desired_option)

    # Find the element and click on it
    WebDriverWait(driver, TIMEOUT).until(
        EC.element_to_be_clickable((By.XPATH, desired_option))
    )

    driver.find_element(by=By.XPATH, value=desired_option).click()

    print(f"Clicked {municipio}")

    categories_xpath = '//*[@id="frm1:SelBoxCategoria"]'

    WebDriverWait(driver, TIMEOUT).until(
        EC.element_to_be_clickable((By.XPATH, categories_xpath))
    )

    try:
        driver.find_element(By.XPATH, categories_xpath).click()
    except Exception as e:
        print(e)
        driver.execute_script(
            "arguments[0].click();", driver.find_element(By.XPATH, categories_xpath)
        )

    time.sleep(2)

    desired_category_xpath = get_desired_category_xpath(driver)
    success = False
    if desired_category_xpath:
        # Click on the desired category
        WebDriverWait(driver, TIMEOUT).until(
            EC.element_to_be_clickable((By.XPATH, desired_category_xpath))
        )
        driver.find_element(by=By.XPATH, value=desired_category_xpath).click()

        print(f"Clicked {desired_category_xpath}")
        success = True

    return success

    # Click on the search button

In [10]:
def get_available_periods(driver):
    periodo_xpath = '//*[@id="frm1:SelBoxPeriodo"]'

    WebDriverWait(driver, TIMEOUT).until(
        EC.element_to_be_clickable((By.XPATH, periodo_xpath))
    )

    # Click on the dropdown menu
    driver.find_element(By.XPATH, periodo_xpath).click()

    soup = BeautifulSoup(driver.page_source, "html.parser")
    periodo_selector = soup.find("select", {"id": "frm1:SelBoxPeriodo"})
    period_options = periodo_selector.find_all("option")

    periods = []
    for option in period_options:
        periods.append(option.text)

    return periods

In [11]:
def periods_for_municipality(municipio, driver):
    """
    Does the whole pipeline to get the periods for a municipality
    """
    go_to_consultas(driver)
    success = select_entidad_and_categoria(municipio, driver)

    if success:
        print("Selected entidad and category")
        time.sleep(3)
        # Get all possible periods
        periods = get_available_periods(driver)
        # Create a dictionary with the period name and its number
        periods_dict = {}
        num_periods = list(range(1, len(periods) + 1))

        for period, num_period in zip(periods, num_periods):
            periods_dict[num_period] = period

    else:
        print("Something went wrong")
        periods_dict = None

    return periods_dict

In [12]:
def get_excel_period(num_period, period_dict, municipio, driver):
    """
    Gets excel file from chip.gov.co
    """
    period_selector = f'//*[@id="frm1:SelBoxPeriodo"]/option[{num_period}]'
    period_name = period_dict[num_period]

    WebDriverWait(driver, TIMEOUT).until(
        EC.element_to_be_clickable((By.XPATH, period_selector))
    ).click()

    time.sleep(2)

    reporte_dropdown = '//*[@id="frm1:SelBoxForma"]'

    WebDriverWait(driver, TIMEOUT).until(
        EC.element_to_be_clickable((By.XPATH, reporte_dropdown))
    ).click()

    soup = BeautifulSoup(driver.page_source, "html.parser")
    # Find class with selboxforma
    reporte_class = soup.find_all("select", {"id": "frm1:SelBoxForma"})
    # Find all options
    reporte_options = reporte_class[0].find_all("option")
    num_option = 1
    for option in reporte_options:
        if option.text == "REPORTE_INFORMACION":
            break
        num_option += 1

    reporte_xpath = f'//*[@id="frm1:SelBoxForma"]/option[{num_option}]'

    # Click on reporte ingresos
    WebDriverWait(driver, TIMEOUT).until(
        EC.element_to_be_clickable((By.XPATH, reporte_xpath))
    ).click()

    consulta_xpath = '//*[@id="frm1:BtnConsular"]'
    WebDriverWait(driver, TIMEOUT).until(
        EC.element_to_be_clickable((By.XPATH, consulta_xpath))
    ).click()

    excel_xpath = '//*[@id="frm1:_t263"]'
    WebDriverWait(driver, TIMEOUT).until(
        EC.element_to_be_clickable((By.XPATH, excel_xpath))
    )
    # Get excel with pandas
    # df = pd.read_excel(excel_url)
    nivel_dropdown = '//*[@id="frm1:SelBoxNivel"]'
    nivel_xpath = '//*[@id="frm1:SelBoxNivel"]/option[10]'

    WebDriverWait(driver, TIMEOUT).until(
        EC.element_to_be_clickable((By.XPATH, nivel_dropdown))
    ).click()

    WebDriverWait(driver, TIMEOUT).until(
        EC.element_to_be_clickable((By.XPATH, nivel_xpath))
    ).click()

    time.sleep(6)

    # wait until page is fully loaded
    WebDriverWait(driver, TIMEOUT).until(
        EC.element_to_be_clickable((By.XPATH, excel_xpath))
    )

    df = pd.read_html(driver.page_source)

    for dataframe in df:
        if len(dataframe.columns) > 5:
            final_df = dataframe
            # Remove unnamed columns
            final_df = final_df.loc[:, ~final_df.columns.str.contains("^Unnamed")]

            # Check if directory exists
            if not os.path.exists(f"{OUTPUT_PATH}/{municipio}"):
                os.makedirs(f"{OUTPUT_PATH}/{municipio}")

            final_df.to_csv(
                f"{OUTPUT_PATH}/{municipio}/INGRESOS_{municipio}_{period_name}.csv",
                index=False,
                encoding="utf-8-sig",
            )
            print(f"INGRESOS_{municipio}_{period_name}.csv saved")
            break

    back_xpath = '//*[@id="frm1:j_idt210"]'
    WebDriverWait(driver, TIMEOUT).until(
        EC.element_to_be_clickable((By.XPATH, back_xpath))
    )
    try:
        driver.find_element(by=By.XPATH, value=back_xpath).click()
    except:
        print("No back button")
        driver.execute_script(
            "arguments[0].click();", driver.find_element(by=By.XPATH, value=back_xpath)
        )

    return final_df

### Pipeline

In [13]:
df = read_from_xls(FILE_PATH)

In [15]:
# Setup selenium webdriver
municipio = "ABEJORRAL"
driver_path = "C:/Users/fdmol/Desktop/chrome-drivers/chromedriver.exe"
service = Service(executable_path=driver_path)
driver = webdriver.Chrome(service=service)

In [16]:
periods_dict = periods_for_municipality(municipio, driver)

Called https://www.chip.gov.co/schip_rt/index.jsf
Entered consulta ciudadana page
2
Max Ratio: 1.0 for ABEJORRAL and ABEJORRAL
//*[@id="frm1:SelBoxEntidadCiudadano_div"]/div/div[2]
Clicked ABEJORRAL
Clicked //*[@id="frm1:SelBoxCategoria"]/option[24]
Selected entidad and category


In [20]:
num_periods = list(periods_dict.keys())
municipio = "ALEJANDRÍA"
for period in num_periods:
    if int(period) <= 1:
        continue
    success = False
    while not success:
        try:
            go_to_consultas(driver)
            success = select_entidad_and_categoria(municipio, driver)
            get_excel_period(period, periods_dict, municipio, driver)
            success = True
            time.sleep(0.25)
        except Exception as e:
            print(e)
            print("Error en periodo: ", period)
            time.sleep(1.5)

Called https://www.chip.gov.co/schip_rt/index.jsf
Entered consulta ciudadana page
2
Max Ratio: 1.0 for ALEJANDRIA and ALEJANDRIA
//*[@id="frm1:SelBoxEntidadCiudadano_div"]/div/div[2]
Clicked ALEJANDRIA
Message: no such element: No node with given id found
  (Session info: chrome=115.0.5790.171); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
Backtrace:
	GetHandleVerifier [0x009D2B63+47795]
	(No symbol) [0x009667E1]
	(No symbol) [0x0086C7ED]
	(No symbol) [0x0085E491]
	(No symbol) [0x0085CCEA]
	(No symbol) [0x0085D39A]
	(No symbol) [0x0085D338]
	(No symbol) [0x00875CE7]
	(No symbol) [0x0086FF36]
	(No symbol) [0x0086F10F]
	(No symbol) [0x00870F4A]
	(No symbol) [0x00870FF0]
	(No symbol) [0x0089DE67]
	(No symbol) [0x0089CC09]
	(No symbol) [0x0089B3DB]
	(No symbol) [0x0089A835]
	(No symbol) [0x00893130]
	(No symbol) [0x008B4CCC]
	(No symbol) [0x00892BC6]
	(No symbol) [0x008B5024]
	(N